## Here we will try our best shot to predict the goals with our features

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Master_UdeM/IFT-6758/Project/ift6758-project-main

/content/gdrive/MyDrive/Master_UdeM/IFT-6758/Project/ift6758-project-main


In [3]:
#!git clone https://GIT_TOKEN@github.com/M0rph3e/ift6758-project-main.git #not anymore
! git pull

Already up to date.


In [12]:
!git status

On branch fengineering2
Your branch is up to date with 'origin/fengineering2'.

nothing to commit, working tree clean


In [54]:
!pip install comet_ml pandas_profiling

In [13]:
! git branch

  best_shot_trial
* fengineering2
  main


# Real **shµt** starts from here

In [14]:
from ift6758.features.feature_engineering2 import SeasonDataSetTwo
from ift6758.data.data_acquisition import Season
from ift6758.visualizations.simple_visualization import Utilities 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import random
import pandas as pd
import pickle
import sys
import json

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train_years = [2015,2016,2017]
valid_years = [2018]
test_years=[2019]


train_dataset = SeasonDataSetTwo(train_years)
valid_dataset = SeasonDataSetTwo(valid_years)
train_df = train_dataset.get_features_2()
valid_df = train_dataset.get_features_2()

100%|██████████| 1299/1299 [07:51<00:00,  2.75it/s]


Len of games_list in 2015 is 1325
Saved new pickle with all events in ../ift6758/data/PICKLE//2015_clean_all_events.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2015.pkl
Len of games_list in 2015 is 1325


100%|██████████| 1299/1299 [07:49<00:00,  2.77it/s]


Len of games_list in 2016 is 1332


In [ ]:
test_data = SeasonDataSet(test_years)
test_df = test_data.get_tidy_data()

File already Exists, loading from ../ift6758/data/PICKLE//2019_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2019_period_info.pkl


# Export the data csv to comet (do not execute for now)

In [ ]:
from comet_ml import Experiment
import os



experiment = Experiment(
    api_key=os.environ.get('COMET_API_KEY'),
    project_name='milestone-2',
    workspace='morph-e',
)

experiment.log_dataframe_profile(
train_df, 
name='train_set_from_FE1',  # keep this name
dataframe_format='csv'  # ensure you set this flag!
)

experiment.log_dataframe_profile(
test_df, 
name='test_set_from_FE1',  # keep this name
dataframe_format='csv'  # ensure you set this flag!
)



COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/morph-e/milestone-2/a119e493bebd462681f1c4eda03a6ad5
COMET INFO:   Others:
COMET INFO:     notebook_url : https://colab.research.google.com/notebook#fileId=https%3A%2F%2Fgithub.com%2FM0rph3e%2Fift6758-project-main%2Fblob%2Fbest_shot_trial%2Fnotebooks%2Ftry_your_best_shot.ipynb
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics

{'dataframe': {'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=0d5ccaa30315475799de3cfc66f94e40&experimentKey=3f87ca7e6bda48558e33fe0218c750cd',
  'assetId': '0d5ccaa30315475799de3cfc66f94e40',
  'web': 'https://www.comet.ml/api/asset/download?assetId=0d5ccaa30315475799de3cfc66f94e40&experimentKey=3f87ca7e6bda48558e33fe0218c750cd'},
 'profile': None}

# Split Data

In [ ]:
features = ['result.secondaryType', 'distanceNet', 'angleNet']
label = ['isGoal']

In [ ]:
X_train = train_df[features]
y_train = train_df[label]

X_valid = valid_df[features]
y_valid = valid_df[label]
X_train, y_train, X_valid, y_valid

(       result.secondaryType  distanceNet   angleNet
 0                Wrist Shot    34.525353  10.007980
 1                 Snap Shot    10.000000   0.000000
 2                 Snap Shot    14.035669  -4.085617
 3                Wrist Shot    49.517674 -46.636577
 4                    Tip-In    28.017851  -2.045408
 ...                     ...          ...        ...
 246798           Wrist Shot    61.326992 -24.057349
 246799            Slap Shot    61.294372 -28.237446
 246800           Wrist Shot     7.615773 -23.198591
 246801           Wrist Shot     7.211103 -33.690068
 246802           Wrist Shot    48.104054  43.315316
 
 [246787 rows x 3 columns],         isGoal
 0            0
 1            0
 2            0
 3            0
 4            0
 ...        ...
 246798       0
 246799       0
 246800       0
 246801       0
 246802       0
 
 [246787 rows x 1 columns],        result.secondaryType  distanceNet   angleNet
 0                Wrist Shot    34.525353  10.007980
 1      

### One Hot encode the shot type (I want to refer on the fact that the shot type affect the goal)

In [ ]:
def one_hot(self, y):
        """Return a one hot encoded array of the labels
        """
        return np.eye(len(self.classes))[np.vectorize(lambda c: self.class_labels[c])(y).reshape(-1)]


In [ ]:
X_train = X_train.rename(columns={'result.secondaryType' : 'shotType'})
X_valid = X_valid.rename(columns={'result.secondaryType' : 'shotType'})

# Replace the NaN for ShotType by a dummy value 'Unknown'

In [ ]:
print(X_train[X_train['shotType'].isna()].count(), X_valid[X_valid['shotType'].isna()].count())
X_train['shotType'] = X_train['shotType'].fillna("Unknown")
X_valid['shotType'] = X_valid['shotType'].fillna("Unknown")

shotType        0
distanceNet    27
angleNet       27
dtype: int64 shotType        0
distanceNet    27
angleNet       27
dtype: int64


In [ ]:
X_train['shotType'].unique()

array(['Wrist Shot', 'Snap Shot', 'Tip-In', 'Backhand', 'Slap Shot',
       'Deflected', 'Wrap-around', 'Unknown'], dtype=object)

In [ ]:
from sklearn import feature_extraction 
vect = feature_extraction.DictVectorizer()


onehots_neighbors_train = vect.fit_transform(X_train[['shotType']].to_dict('records'))
onehots_neighbors_train = pd.DataFrame(onehots_neighbors_train.toarray()).astype('int').add_prefix('Shot_')

onehots_neighbors_valid = vect.fit_transform(X_valid[['shotType']].to_dict('records'))
onehots_neighbors_valid = pd.DataFrame(onehots_neighbors_valid.toarray()).astype('int').add_prefix('Shot_')

In [ ]:
onehots_neighbors_valid.head()

,Shot_0,Shot_1,Shot_2,Shot_3,Shot_4,Shot_5,Shot_6,Shot_7
0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0


In [ ]:
#concat to X_train and X_valid
X_train = pd.concat([X_train, onehots_neighbors_train], axis=1)
X_valid = pd.concat([X_valid, onehots_neighbors_valid], axis=1)

In [ ]:
#drop shot type
X_train = X_train.drop(columns='shotType')
X_valid = X_valid.drop(columns='shotType')
X_valid

,distanceNet,angleNet,Shot_0,Shot_1,Shot_2,Shot_3,Shot_4,Shot_5,Shot_6,Shot_7
0,34.525353,10.007980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,10.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,14.035669,-4.085617,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,49.517674,-46.636577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,28.017851,-2.045408,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
246798,61.326992,-24.057349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246799,61.294372,-28.237446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246800,7.615773,-23.198591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246801,7.211103,-33.690068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Try Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [10 ,30, 50, 75, 100],
     'max_depth': [2, 3, 5, 8, 10, 12]
}

rf = RandomForestClassifier(criterion='entropy')
sh = GridSearchCV(rf,param_grid, cv=4)
model = sh.fit(X_train,y_train)